# Doc to get seldon core setup

In [ ]:
!kind create cluster --config=kind.yaml

curl -L https://istio.io/downloadIstio | sh -

In [ ]:
Install Minio

In [1]:
!helm repo add minio https://helm.min.io/
!helm install minio minio/minio \
    --set accessKey=minioadmin \
    --set secretKey=minioadmin \
    --namespace seldon-system

"minio" already exists with the same configuration, skipping
NAME: minio
LAST DEPLOYED: Tue Dec 20 13:56:36 2022
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Minio can be accessed via port 9000 on the following DNS name from within your cluster:
minio.seldon-system.svc.cluster.local

To access Minio from localhost, run the below commands:

  1. export POD_NAME=$(kubectl get pods --namespace seldon-system -l "release=minio" -o jsonpath="{.items[0].metadata.name}")

  2. kubectl port-forward $POD_NAME 9000 --namespace seldon-system

Read more about port forwarding here: http://kubernetes.io/docs/user-guide/kubectl/kubectl_port-forward/

You can now access Minio server on http://localhost:9000. Follow the below steps to connect to Minio server with mc client:

  1. Download the Minio mc client - https://docs.minio.io/docs/minio-client-quickstart-guide

  2. Get the ACCESS_KEY=$(kubectl get secret minio -o jsonpath="{.data.accesskey}" | base64 --decode) and

In [ ]:
Install Core

In [ ]:
!helm upgrade seldon-core seldon-charts/seldon-core-operator \
    -f core-values.yaml \
    --version 1.15.0 \
    --namespace seldon-system \
    --set usageMetrics.enabled=true \
    --set istio.enabled=true \
    --install

In [ ]:
/seldon/<namespace>/<model-name>/api/v1.0/doc/

/seldon/seldon/sklearn/api/v1.0/doc/




In [ ]:
Access Core

In [ ]:
!kubectl port-forward -n istio-system svc/istio-ingressgateway 8080:80


In [10]:
import requests

URL = f"http://localhost:8080/seldon/default/sklearn/api/v1.0/predictions"
headers = {
    "content-type": "application/json"
}
prediction = {
    "data": {
        "ndarray": [[1, 2, 3, 4]]
    }
}


r = requests.post(URL, headers=headers, json=prediction)
r.text


'{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162835,0.00366803903943666,0.995633441507447]]},"meta":{"requestPath":{"classifier":"seldonio/sklearnserver:1.15.0"}}}\n'

Setup Analytics

## Prometheus Operator

Follow these instructions

[Install Prometheus Operator with Seldon](https://docs.seldon.io/projects/seldon-core/en/latest/analytics/analytics.html)

In [ ]:
!kubectl create namespace seldon-monitoring

In [ ]:
!helm upgrade --install prometheus kube-prometheus \
    --version 8.0.9 \
    --namespace seldon-system \
    --values values-kube-prometheus.yaml \
    --repo https://charts.bitnami.com/bitnami


In [ ]:
!kubectl rollout status -n seldon-monitoring statefulsets/prometheus-seldon-monitoring-prometheu

In [ ]:
Grafana

Install graphana properlly from docs

